https://towardsdatascience.com/understanding-topic-coherence-measures-4aa41339634c

In [9]:
idCuda = 3
idTorch = 0

In [11]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = f"{str(idCuda)}"

In [10]:
import torch
import pandas as pd

In [11]:
torch.cuda.device_count()

4

In [12]:
torch.cuda.empty_cache()
torch.cuda.set_device(idTorch)

if torch.cuda.is_available():
    GPU = torch.cuda.get_device_name(idTorch)
    print(f"PyTorch available GPU: {GPU}")
else:
    print("GPU not available")

PyTorch available GPU: NVIDIA RTX A5000


In [13]:
device = torch.device(f"cuda:{idTorch}" if torch.cuda.is_available() else "cpu")
print(f'Device: {device}')

Device: cuda:0


In [14]:
# device = torch.device("cpu")
# print(f'Device: {device}')

In [15]:
data = pd.read_csv('/home/patrick.araujo/commentsGooglePlay/output/spotify5_balanced_output.csv')

In [16]:
data

,level_0,index,Time_submitted,content,score,Total_thumbsup,Reply,sentiment,lengthContent,Language,scoreSA,reviewId
0,49179,49436,2022-02-17 07:33:36,Songs stop playing for no reason. This is one ...,2,0,NaN,0,116,en,1,22701
1,30989,31215,2022-04-17 22:10:35,Very good app I've been able to listen to musi...,5,1,NaN,2,105,en,4,3761
2,34086,34321,2022-04-13 16:40:13,Something really frustrating that happens almo...,4,27,NaN,2,433,en,3,4004
3,21919,22078,2022-05-09 20:31:13,Unlimited music for a great price. What else d...,5,0,NaN,2,57,en,4,10125
4,58110,58382,2022-01-13 13:27:20,Spotify is awesome app for listening music but...,4,0,NaN,2,181,en,3,32559
...,...,...,...,...,...,...,...,...,...,...,...,...
33750,29036,29254,2022-04-21 03:36:32,"This is a very good app, I can easily access t...",4,0,NaN,2,61,en,3,21174
33751,16686,16817,2022-05-26 10:58:34,The music quality is excellent.,4,0,NaN,2,31,en,3,8162
33752,42797,43048,2022-03-12 06:06:03,"Great for listening to all kinds of music, but...",4,1,NaN,2,242,en,3,1020
33753,47446,47701,2022-02-24 05:39:11,Every time I say hey Spotify and play a song i...,3,0,NaN,1,214,en,2,27340


In [5]:
classe = 4

sentiment = False

In [17]:
input = data

In [38]:
if sentiment:
    input = data[data['sentiment'] == classe]
else:
    input = data[data['score'] == classe]

In [18]:
input

,level_0,index,Time_submitted,content,score,Total_thumbsup,Reply,sentiment,lengthContent,Language,scoreSA,reviewId
0,49179,49436,2022-02-17 07:33:36,Songs stop playing for no reason. This is one ...,2,0,NaN,0,116,en,1,22701
1,30989,31215,2022-04-17 22:10:35,Very good app I've been able to listen to musi...,5,1,NaN,2,105,en,4,3761
2,34086,34321,2022-04-13 16:40:13,Something really frustrating that happens almo...,4,27,NaN,2,433,en,3,4004
3,21919,22078,2022-05-09 20:31:13,Unlimited music for a great price. What else d...,5,0,NaN,2,57,en,4,10125
4,58110,58382,2022-01-13 13:27:20,Spotify is awesome app for listening music but...,4,0,NaN,2,181,en,3,32559
...,...,...,...,...,...,...,...,...,...,...,...,...
33750,29036,29254,2022-04-21 03:36:32,"This is a very good app, I can easily access t...",4,0,NaN,2,61,en,3,21174
33751,16686,16817,2022-05-26 10:58:34,The music quality is excellent.,4,0,NaN,2,31,en,3,8162
33752,42797,43048,2022-03-12 06:06:03,"Great for listening to all kinds of music, but...",4,1,NaN,2,242,en,3,1020
33753,47446,47701,2022-02-24 05:39:11,Every time I say hey Spotify and play a song i...,3,0,NaN,1,214,en,2,27340


In [ ]:
from transformers.pipelines import pipeline
from bertopic import BERTopic
from transformers import pipeline
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
# from umap import UMAP
# from hdbscan import HDBSCAN

# we add this to remove stopwords, for lower volumes of data stopwords can cause issues
vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words="english")

ctfidf_model = ClassTfidfTransformer()
embedding_model = pipeline("feature-extraction", model="bert-base-uncased", device=device)
representation_model = KeyBERTInspired()
# umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')
# hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# deal with df if needed
if type(input['content']) is list:
    text = input['content']
else:
    text = input['content'].tolist()

model = BERTopic(
    embedding_model=embedding_model, 
    vectorizer_model=vectorizer_model, 
    ctfidf_model=ctfidf_model, 
    # representation_model=representation_model, 
    top_n_words=25,
    # nr_topics = None,
    nr_topics = "auto",
    min_topic_size = 2,
    language='english', calculate_probabilities=True,
    verbose=True
)

topics, probs = model.fit_transform(text)

2024-03-24 10:39:41,358 - BERTopic - Embedding - Transforming documents to embeddings.
100%|████████████████████████████████████| 33755/33755 [05:11<00:00, 108.32it/s]
2024-03-24 10:44:53,183 - BERTopic - Embedding - Completed ✓
2024-03-24 10:44:53,184 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-24 10:45:35,906 - BERTopic - Dimensionality - Completed ✓
2024-03-24 10:45:35,910 - BERTopic - Cluster - Start clustering the reduced embeddings


In [ ]:
# Initialize an empty DataFrame with the desired column names
export = pd.DataFrame(columns=['comment', 'reviewId', 'sentiment', 'numTopic'])

for i in range(len(topics)):
    # Create a new DataFrame row with the current iteration's data
    new_row = {'comment': input[
               
               'content'].iloc[i], 'reviewId': input['reviewId'].iloc[i], 'sentiment': input['sentiment'].iloc[i], 'numTopic': topics[i]}
    
    # Append the new row to the export DataFrame, ignoring the index to avoid index duplication
    export = pd.concat([export, pd.DataFrame([new_row])], ignore_index=True)

    print(f"Comment {i+1}: \"{input['content'].iloc[i]}\"\t-\tTopic ({topics[i]})")
input = pd.merge(left=input, right=export[['reviewId', 'numTopic']], on='reviewId')

In [ ]:
input

In [ ]:
export

In [ ]:
all_topics = model.get_topics()

In [ ]:
all_topics

In [ ]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from gensim import corpora

def calculate_coherence_score(topic_model, docs):
    # Preprocess documents
    cleaned_docs = topic_model._preprocess_text(docs)

    # Extract vectorizer and tokenizer from BERTopic
    vectorizer = topic_model.vectorizer_model
    tokenizer = vectorizer.build_tokenizer()

    # Extract features for Topic Coherence evaluation
    words = vectorizer.get_feature_names_out()
    # depending on the version and if you get an error use commented out code below:
    # words = vectorizer.get_feature_names()
    tokens = [tokenizer(doc) for doc in cleaned_docs]
    dictionary = corpora.Dictionary(tokens)
    corpus = [dictionary.doc2bow(token) for token in tokens]
    # Create topic words
    topic_words = [[dictionary.token2id[w] for w in words if w in dictionary.token2id]
    for _ in range(len(topic_model.get_topics()))]
    
    # this creates a list of the token ids (in the format of integers) of the words in words that are also present in the 
    # dictionary created from the preprocessed text. The topic_words list contains list of token ids for each 
    # topic.

    coherence_model = CoherenceModel(topics=topic_words,
                                    texts=tokens,
                                    corpus=corpus,
                                    dictionary=dictionary,
                                    coherence='c_v')
    coherence = coherence_model.get_coherence()

    return coherence

In [ ]:
coherence_score = calculate_coherence_score(model, text)
print("Coherence Score:", coherence_score)

In [ ]:
topics = pd.DataFrame(columns=['id', 'topics'])

for topic, word_probs in all_topics.items():
    words = [word for word, _ in word_probs]
    new_row = {'id': topic, 'topics': words}
    topics = pd.concat([topics, pd.DataFrame([new_row])], ignore_index=True)

In [ ]:
topics

# Topics and comments

In [ ]:
input = pd.merge(left=input, right=topics, left_on='numTopic', right_on='id', how ='left')

In [ ]:
input


In [ ]:
input = input.drop('numTopic', axis=1)
input = input.drop('id', axis=1)

tipo = '5'
if sentiment:
    tipo = '3'
    topics['sentiment'] = classe
else:
    topics['score'] = classe
input.to_csv(f'export/export{tipo}_classe{classe}.csv', index=False)

In [ ]:
topics

In [ ]:
topics.to_csv(f'out/outAuto.csv', index=False)

In [ ]:
topics.to_csv(f'out/out{tipo}_classe{classe}.csv', index=False)

Termina aqui